In [3]:
import os
import sys
import pandas as pd
import numpy as np
sys.path.append(os.path.abspath(os.path.join('../scripts')))
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.express as px

In [4]:
from data_generator import vis_train_features

In [6]:
# extract label and audio features for a single training example
vis_text, vis_raw_audio, vis_mfcc_feature, vis_spectrogram_feature, vis_audio_path = vis_train_features()

FileNotFoundError: [Errno 2] No such file or directory: 'train_corpus.json'

## Train 

In [7]:
from sample_models import *
# import function for training acoustic model
from train_utils import train_model

## Simple RNN

In [8]:
model_0 = simple_rnn_model(input_dim=13)

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       [(None, None, 13)]        0         
_________________________________________________________________
rnn (GRU)                    (None, None, 48)          9072      
_________________________________________________________________
softmax (Activation)         (None, None, 48)          0         
Total params: 9,072
Trainable params: 9,072
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
train_model(input_to_softmax=model_0, 
            pickle_path='model_0.pickle', 
            save_model_path='model_0.h5',
            spectrogram=False)

FileNotFoundError: [Errno 2] No such file or directory: 'train_corpus.json'

In [10]:
model_1 = rnn_model(input_dim=13, 
                    units=200,
                    activation='relu')

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       [(None, None, 13)]        0         
_________________________________________________________________
rnn (GRU)                    (None, None, 200)         129000    
_________________________________________________________________
batch_normalization (BatchNo (None, None, 200)         800       
_________________________________________________________________
time_distributed (TimeDistri (None, None, 48)          9648      
_________________________________________________________________
softmax (Activation)         (None, None, 48)          0         
Total params: 139,448
Trainable params: 139,048
Non-trainable params: 400
_________________________________________________________________
None


In [11]:
train_model(input_to_softmax=model_1, 
            pickle_path='model_1.pickle', 
            save_model_path='model_1.h5',
            spectrogram=False)

FileNotFoundError: [Errno 2] No such file or directory: 'train_corpus.json'

In [12]:
model_2 = cnn_rnn_model(input_dim=13, 
                        filters=200,
                        kernel_size=11, 
                        conv_stride=2,
                        conv_border_mode='valid',
                        units=200)

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       [(None, None, 13)]        0         
_________________________________________________________________
conv1d (Conv1D)              (None, None, 200)         28800     
_________________________________________________________________
bn_conv_1d (BatchNormalizati (None, None, 200)         800       
_________________________________________________________________
rnn (SimpleRNN)              (None, None, 200)         80200     
_________________________________________________________________
bn_norm (BatchNormalization) (None, None, 200)         800       
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 48)          9648      
_________________________________________________________________
softmax (Activation)         (None, None, 48)         

In [13]:
train_model(input_to_softmax=model_2, 
            pickle_path='model_2.pickle', 
            save_model_path='model_2.h5', 
            spectrogram=False)

FileNotFoundError: [Errno 2] No such file or directory: 'train_corpus.json'

In [14]:
model_3 = deep_rnn_model(input_dim=161,
                         units=200,
                         recur_layers=2)

+Layer 1
Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       [(None, None, 161)]       0         
_________________________________________________________________
rnn (SimpleRNN)              (None, None, 200)         72400     
_________________________________________________________________
bn_norm (BatchNormalization) (None, None, 200)         800       
_________________________________________________________________
rnn1 (GRU)                   (None, None, 200)         241200    
_________________________________________________________________
bnorm1 (BatchNormalization)  (None, None, 200)         800       
_________________________________________________________________
time_distributed_2 (TimeDist (None, None, 48)          9648      
_________________________________________________________________
softmax (Activation)         (None, None, 48)

In [15]:
train_model(input_to_softmax=model_3, 
            pickle_path='model_3.pickle', 
            save_model_path='model_3.h5', 
            spectrogram=True)

FileNotFoundError: [Errno 2] No such file or directory: 'train_corpus.json'

In [16]:
model_4 = bidirectional_rnn_model(input_dim=161, 
                                  units=200)

Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       [(None, None, 161)]       0         
_________________________________________________________________
bidirectional (Bidirectional (None, None, 400)         579200    
_________________________________________________________________
bnorm (BatchNormalization)   (None, None, 400)         1600      
_________________________________________________________________
time_distributed_3 (TimeDist (None, None, 48)          19248     
_________________________________________________________________
softmax (Activation)         (None, None, 48)          0         
Total params: 600,048
Trainable params: 599,248
Non-trainable params: 800
_________________________________________________________________
None


In [17]:
train_model(input_to_softmax=model_4, 
            pickle_path='model_4.pickle', 
            save_model_path='model_4.h5', 
            spectrogram=True)

FileNotFoundError: [Errno 2] No such file or directory: 'train_corpus.json'

In [18]:
import os
import mlflow
from getpass import getpass

os.environ['MLFLOW_TRACKING_USERNAME'] = input('Enter your DAGsHub username: ')
os.environ['MLFLOW_TRACKING_PASSWORD'] = getpass('Enter your DAGsHub access token: ')
os.environ['MLFLOW_TRACKING_PROJECTNAME'] = input('Enter your DAGsHub project name: ')

mlflow.set_tracking_uri(f'https://dagshub.com/' + os.environ['MLFLOW_TRACKING_USERNAME'] 
                        + '/' + os.environ['MLFLOW_TRACKING_PROJECTNAME'] + '.mlflow')


Enter your DAGsHub username: Desmond Onam
Enter your DAGsHub access token: ········
Enter your DAGsHub project name: Desmond Onam


In [19]:
from glob import glob
import numpy as np
import _pickle as pickle
import seaborn as sns
import mlflow
import mlflow.tensorflow
import matplotlib.pyplot as plt
%matplotlib inline
sns.set_style(style='white')

with mlflow.start_run(run_name="speech-recognition",nested=True):

    # obtain the paths for the saved model history
    all_pickles = sorted(glob("results/*.pickle"))
    # extract the name of each model
    model_names = [item[8:-7] for item in all_pickles]
    # extract the loss history for each model
    valid_loss = [pickle.load( open( i, "rb" ) )['val_loss'] for i in all_pickles]
    train_loss = [pickle.load( open( i, "rb" ) )['loss'] for i in all_pickles]
    # save the number of epochs used to train each model
    num_epochs = [len(valid_loss[i]) for i in range(len(valid_loss))]

    fig = plt.figure(figsize=(16,5))

    # plot the training loss vs. epoch for each model
    ax1 = fig.add_subplot(121)
    for i in range(len(all_pickles)):
        ax1.plot(np.linspace(1, num_epochs[i], num_epochs[i]), 
                train_loss[i], label=model_names[i])
    # clean up the plot
    ax1.legend()  
    ax1.set_xlim([1, max(num_epochs)])
    plt.xlabel('Epoch')
    plt.ylabel('Training Loss')

    # plot the validation loss vs. epoch for each model
    ax2 = fig.add_subplot(122)
    for i in range(len(all_pickles)):
        ax2.plot(np.linspace(1, num_epochs[i], num_epochs[i]), 
                valid_loss[i], label=model_names[i])
    # clean up the plot
    ax2.legend()  
    ax2.set_xlim([1, max(num_epochs)])
    plt.xlabel('Epoch')
    plt.ylabel('Validation Loss')
    #     plt.show()
    plt.savefig("loss.png")
    mlflow.log_artifact("loss.png")
    plt.show()

ChunkedEncodingError: ("Connection broken: InvalidChunkLength(got length b'', 0 bytes read)", InvalidChunkLength(got length b'', 0 bytes read))

In [20]:
import IPython
display(IPython.display.IFrame("https://dagshub.com/"+ os.environ['MLFLOW_TRACKING_USERNAME'] 
                        + '/' + os.environ['MLFLOW_TRACKING_PROJECTNAME'] + "/experiments/#/",'100%',600))

In [21]:
model_end = final_model(input_dim=13, units=200)

Model: "functional_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       [(None, None, 13)]        0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, None, 400)         342400    
_________________________________________________________________
bnorm1 (BatchNormalization)  (None, None, 400)         1600      
_________________________________________________________________
bidirectional_2 (Bidirection (None, None, 400)         961600    
_________________________________________________________________
bnorm2 (BatchNormalization)  (None, None, 400)         1600      
_________________________________________________________________
time_distributed_4 (TimeDist (None, None, 48)          19248     
_________________________________________________________________
softmax (Activation)         (None, None, 48)        

In [22]:
import mlflow
mlflow.tensorflow.autolog()
mlflow.log_param("task",2)

MlflowException: API request to endpoint /api/2.0/mlflow/runs/create failed with error code 404 != 200. Response body: '<!DOCTYPE html>
<html lang="en-US">
  <head data-suburl="">
    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8"/>
    <meta http-equiv="X-UA-Compatible" content="IE=edge"/>

    
      <meta name="viewport" content="width=device-width, initial-scale=1"/>
    
    <meta name="referrer" content="no-referrer"/>
    <meta name="_csrf" content="uUozbnue37_gBJv_yFisRoDWKPw6MTYyODc4MDg3MDI0NDEwNjk3MA=="/>
    <meta name="_suburl" content=""/>
    
    <meta name="_bareRepo" content=""/>
    <meta name="_demoRepoURL" content="/nirbarazida/Pneumonia-Classification"/>

    
    
  <script>(function(w, d, s, l, i) {
      w[l] = w[l] || [];
      w[l].push({
        'gtm.start':
          new Date().getTime(), event: 'gtm.js'
      });
      var f = d.getElementsByTagName(s)[0],
        j = d.createElement(s),
        dl = l != 'dataLayer' ? '&l=' + l : '';
  

In [23]:
train_model(input_to_softmax=model_end, 
            pickle_path='model_end.pickle', 
            save_model_path='model_end.h5', 
            spectrogram=False) # change to False if you would like to use MFCC features

FileNotFoundError: [Errno 2] No such file or directory: 'train_corpus.json'